In [1]:
import os, random
import numpy as np
import zarr, copick
from tqdm import tqdm
from monai.data import DataLoader, CacheDataset, decollate_batch
from monai.transforms import (
    Compose, 
    EnsureChannelFirstd, 
    ScaleIntensityRanged, 
    CropForegroundd, 
    Orientationd, 
    Spacingd, 
    EnsureTyped, 
    Activations, 
    AsDiscrete, 
    Resized, 
    RandFlipd, 
    RandRotate90d, 
    RandZoomd,
    RandGridPatchd,
    NormalizeIntensityd,
    RandCropByLabelClassesd,
    Resized, 
    RandZoomd,
    Activations, 
    CropForegroundd, 
    ScaleIntensityRanged, 
    RandCropByPosNegLabeld,    
)
from monai.networks.nets import UNet
from monai.losses import DiceLoss, FocalLoss, TverskyLoss
from monai.metrics import DiceMetric, ConfusionMatrixMetric
# from skimage
import mlflow
import mlflow.pytorch
from sklearn.model_selection import train_test_split

from monai.transforms import Compose, EnsureChannelFirstd, NormalizeIntensityd, Orientationd, RandCropByLabelClassesd, RandRotate90d, RandFlipd
from monai.data import CacheDataset, DataLoader, Dataset
import mlflow, optuna, torch

/mnt/jonathan/2024_czii_mlchallenge_notebooks/pyUNET/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
copick_config_path = "../../../simulations/ml_challenge/ml_config.json"
root = copick.from_file(copick_config_path)

In [3]:
def get_tomogram_array(copick_run, voxel_spacing=10, tomo_type='wbp'):
    voxel_spacing_obj = copick_run.get_voxel_spacing(voxel_spacing)
    tomogram = voxel_spacing_obj.get_tomogram(tomo_type)
    image = zarr.open(tomogram.zarr(), mode='r')['0']
    return image[:]

def get_segmentation_array(copick_run, segmentation_name, voxel_spacing=10, is_multilabel=True):
    #seg_memb = copick_run.get_segmentations(name="membrane")
    seg_memb = copick_run.get_segmentations()
    seg = copick_run.get_segmentations(is_multilabel=is_multilabel, name=segmentation_name, voxel_size=voxel_spacing)
    if len(seg) == 0:
        raise ValueError(f"No segmentations found for session '{session_id}' and segmentation type '{segmentation_type}'.")
        
    segmentation = zarr.open(seg[0].zarr().path, mode="r")['0'][:]
    _, array = list(zarr.open(seg_memb[0].zarr()).arrays())[0]
    seg_membrane = np.array(array[:])
    #seg_membrane = zarr.open(seg_memb[0].zarr().path, mode="r")['0'][:]
    segmentation[seg_membrane==1] = 1
    return segmentation

In [4]:
voxel_spacing = 10
tomo_type = "wbp"
painting_segmentation_name = "segmentation" #"remotetargets"

runIDs = [run.name for run in root.runs]
data_dicts = []
for runID in tqdm(runIDs):
    run = root.get_run(str(runID))
    tomogram = get_tomogram_array(run)
    segmentation = get_segmentation_array(run, painting_segmentation_name)
    data_dicts.append({"image": tomogram, "label": segmentation})

100%|██████████| 24/24 [00:11<00:00,  2.01it/s]


In [5]:
my_num_samples = 16
train_batch_size = 1
val_batch_size = 1

test_files = data_dicts[-2:]
train_files, val_files = train_test_split(data_dicts[:-2], test_size=0.3)
print(f"Number of training samples: {len(train_files)}")
print(f"Number of validation samples: {len(val_files)}")

# Non-random transforms to be cached
non_random_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image", "label"], axcodes="RAS")
])

# Random transforms to be applied during training
random_transforms = Compose([
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[96, 96, 96],
        num_classes=8,
        num_samples=my_num_samples
    ),
    RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),    
])

# Create the cached dataset with non-random transforms
train_ds = CacheDataset(data=train_files, transform=non_random_transforms, cache_rate=1.0)

# Wrap the cached dataset to apply random transforms during iteration
train_ds = Dataset(data=train_ds, transform=random_transforms)

# DataLoader remains the same
train_loader = DataLoader(
    train_ds,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)

# Validation transforms
val_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[96, 96, 96],
        num_classes=8,
        num_samples=my_num_samples,  # Use 1 to get a single, consistent crop per image
    ),
])

# Create validation dataset
val_ds = CacheDataset(data=val_files, transform=non_random_transforms, cache_rate=1.0)

# Wrap the cached dataset to apply random transforms during iteration
val_ds = Dataset(data=val_ds, transform=random_transforms)

# Create validation DataLoader
val_loader = DataLoader(
    val_ds,
    batch_size=val_batch_size,
    num_workers=4,
    pin_memory=torch.cuda.is_available(),
    shuffle=False,  # Ensure the data order remains consistent
)

Number of training samples: 15
Number of validation samples: 7


Loading dataset: 100%|██████████| 7/7 [00:00<00:00, 23.96it/s]


In [6]:
#loss_function = DiceLoss(include_background=True, to_onehot_y=True, softmax=True)  # softmax=True for multiclass
loss_function = TverskyLoss(include_background=True, to_onehot_y=True, softmax=True)  

dice_metric = DiceMetric(include_background=False, reduction="mean", ignore_empty=True)  # must use onehot for multiclass
recall_metric = ConfusionMatrixMetric(include_background=False, metric_name="recall", reduction="None")

In [7]:
def stack_patches(data):
    shape = data.shape
    new_shape = (shape[0] * shape[1],) + shape[2:]
    return data.view(new_shape)

post_pred = AsDiscrete(argmax=True, to_onehot=len(root.pickable_objects)+1)
post_label = AsDiscrete(to_onehot=len(root.pickable_objects)+1)

def train(train_loader, 
        val_loader, 
        model, 
        device,
        loss_function, 
        metrics_function, 
        optimizer, 
        max_epochs=100,
        val_interval = 2):

    best_f1_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []

    # Initialize tqdm around the epoch loop
    for epoch in tqdm(range(max_epochs), desc="Training Progress", unit="epoch"):

        model.train()
        epoch_loss = 0
        step = 0
        for batch_data in train_loader:
            step += 1
            inputs = batch_data["image"].to(device)  # Shape: [B, C, H, W, D]
            labels = batch_data["label"].to(device)  # Shape: [B, C, H, W, D]            
            optimizer.zero_grad()
            outputs = model(inputs)    # Output shape: [B, num_classes, H, W, D]
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            # Update running epoch loss
            epoch_loss += loss.item()
            avg_loss = epoch_loss / step
        
        # Compute and log average epoch loss
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        mlflow.log_metric("train_loss", epoch_loss, step=epoch+1)

        # Update tqdm description with dynamic loss for each epoch
        tqdm.write(f"Epoch {epoch + 1}/{max_epochs}, avg_train_loss: {avg_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()
            
            with torch.no_grad():
                for val_data in val_loader:
                    val_inputs = val_data["image"].to(device)
                    val_labels = val_data["label"].to(device)
                    val_outputs = model(val_inputs)
                    
                    # Decollate batches into lists
                    val_outputs_list = decollate_batch(val_outputs)
                    val_labels_list = decollate_batch(val_labels)
                    
                    # Apply post-processing
                    metric_val_outputs = [post_pred(i) for i in val_outputs_list]
                    metric_val_labels = [post_label(i) for i in val_labels_list]
                    
                    # Compute metrics
                    metrics_function(y_pred=metric_val_outputs, y=metric_val_labels)

                # Contains recall, precision, and f1 for each class
                metric_values = metrics_function.aggregate(reduction='mean_batch')

                # Log per-class metrics
                for i, (rec, prec, f1) in enumerate(zip(metric_values[0], metric_values[1], metric_values[2])):
                    mlflow.log_metric(f"recall_class_{i+1}", rec.item(), step=epoch + 1)
                    mlflow.log_metric(f"precision_class_{i+1}", prec.item(), step=epoch + 1)
                    mlflow.log_metric(f"f1_score_class_{i+1}", f1.item(), step=epoch + 1)

                # Log average metrics across all classes
                avg_recall =  metric_values[0].mean()
                avg_precision =  metric_values[1].mean()
                avg_f1 =  metric_values[2].mean()                                

                mlflow.log_metric("avg_recall", avg_recall, step=epoch + 1)
                mlflow.log_metric("avg_precision", avg_precision, step=epoch + 1)
                mlflow.log_metric("avg_f1_score", avg_f1, step=epoch + 1)

                tqdm.write(f"Epoch {epoch + 1}/{max_epochs}, avg_recall: {avg_recall:.4f}")        
                metrics_function.reset()

                # Track the best F1 score (mean across all classes)
                if avg_f1 > best_f1_metric:
                    best_f1_metric = avg_f1

                # metric_values.append(metric)
                # if metric > best_metric:
                #     best_metric = metric
                #     best_metric_epoch = epoch + 1
                #     torch.save(model.state_dict(), os.path.join('./', "best_metric_model.pth"))
                    
                #     print("saved new best metric model")
                # print(
                #     f"current epoch: {epoch + 1} current mean recall per class: {', '.join(metric_per_class)}"
                #     f"\nbest mean recall: {best_metric:.4f}"
                #     f"at epoch: {best_metric_epoch}"
                # )
    return best_f1_metric

In [8]:
def set_seed(seed):
    # Set the seed for Python's random module
    random.seed(seed)

    # Set the seed for NumPy
    np.random.seed(seed)

    # Set the seed for PyTorch (both CPU and GPU)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # If using multi-GPU

    # Ensure reproducibility of operations by disabling certain optimizations
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [10]:
def objective(trial,  
              n_classes,
              train_loader, 
              val_loader, 
              loss_function,
              metrics_function, 
              epochs,
              random_seed = 42,
              gpu_count = 1):

    set_seed(random_seed)

    # Assign each trial to a specific GPU based on the trial number
    if gpu_count > 1:
        gpu_id = trial.number % gpu_count  # Cycle through available GPUs
        device = torch.device(f"cuda:{gpu_id}")
        torch.cuda.set_device(device)  # Set the current GPU for this trial
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # Set a unique run name for each trial
    trial_num = f"trial_{trial.number}"

    # Start a new MLflow run for each trial
    with mlflow.start_run(run_name = trial_num, nested=True):  # Nested=True allows it to be part of the overall experiment run
        
        # Sample number of channels
        num_layers = trial.suggest_int("num_layers", 3, 5)
        
        # Generate increasing channels based on 16, 32, 64
        base_channel = trial.suggest_categorical("base_channel", [8, 16, 32, 64])
        channels = [base_channel * (2 ** i) for i in range(num_layers)]

        # Strides pattern
        # Sample number of downsampling layers (those with stride of 2)
        num_downsampling_layers = trial.suggest_int("num_downsampling_layers", 1, num_layers - 1)

        # Define strides: first num_downsampling_layers with stride 2, and the rest with stride 1
        strides_pattern = [2] * num_downsampling_layers + [1] * (num_layers - num_downsampling_layers - 1)        

        # Number of residual units
        num_res_units = trial.suggest_int("num_res_units", 1, 3)

        print('Current Parameters: ')
        print(f'Num Layers: {num_layers}')
        print(f'Num Res Units: {num_res_units}')
        print(f'Channels: {channels}')
        print(f'Num Downsampling Layers: {num_downsampling_layers}')
        print(f'Strides: {strides_pattern}\n')

        # Log parameters to MLflow at each trial
        mlflow.log_params({
            "num_layers": num_layers,
            "base_channel": base_channel,
            "channels": channels,
            "num_downsampling_layers": num_downsampling_layers,
            "strides_pattern": strides_pattern,
            "num_res_units": num_res_units,
            "random_seed": random_seed
        })

        # Now use channels, strides, and num_res_units in your model definition
        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=n_classes,
            channels=channels,
            strides=strides_pattern,
            num_res_units=num_res_units,
        ).to(device)

        # # Sample learning rate using Optuna
        # learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)

        # Define your loss and optimizer, and return the objective (e.g., validation loss)
        lr = 1e-3
        optimizer = torch.optim.Adam(model.parameters(), lr)

        score = train(train_loader, val_loader, 
                      model, device, loss_function, 
                      metrics_function, 
                      optimizer, max_epochs=epochs)

        # Log the score (e.g., validation loss or F1 score) for each trial
        mlflow.log_metric("score", score)

    return score

#### Multi-GPU Exploration

In [12]:
epochs = 100
num_trials = 10
nclasses = len(root.pickable_objects)+1
loss_function = TverskyLoss(include_background=True, to_onehot_y=True, softmax=True)  
my_metrics = ConfusionMatrixMetric(include_background=False, metric_name=["recall",'precision','f1 score'], reduction="none")

# Get available GPUs (for example, on an 8 GPU node)
gpu_count = torch.cuda.device_count()

print(f'Running Architecture Search Over {gpu_count} GPUS\n')
mlflow.set_experiment('unet-model-search')
with mlflow.start_run():
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, nclasses, train_loader, val_loader, 
                                           loss_function, my_metrics, epochs), 
                   n_trials=num_trials) # Run trials on multiple GPUs

    print(f"Best trial: {study.best_trial.value}")
    print(f"Best params: {study.best_params}")

    # Log the best trial in MLflow
    mlflow.log_metric("best_trial_value", study.best_trial.value)
    mlflow.log_params(study.best_trial.params)

[I 2024-10-17 22:18:19,890] A new study created in memory with name: no-name-c8781863-c0ce-4ae5-975f-a466f438b124


Running Architecture Search Over 4 GPUS

Current Parameters: 
Num Layers: 5
Num Res Units: 1
Channels: [32, 64, 128, 256, 512]
Num Downsampling Layers: 2
Strides: [2, 2, 1, 1]



Training Progress:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training Progress:   1%|          | 1/100 [00:09<15:43,  9.53s/epoch]

Epoch 1/100, avg_train_loss: 0.9320


...

Epoch 2/100, avg_train_loss: 0.8873


Training Progress:   2%|▏         | 2/100 [00:25<21:28, 13.15s/epoch]

Epoch 2/100, avg_recall: 0.1397


Training Progress:   2%|▏         | 2/100 [00:30<24:30, 15.01s/epoch]
[W 2024-10-17 22:18:50,094] Trial 0 failed with parameters: {'num_layers': 5, 'base_channel': 32, 'num_downsampling_layers': 2, 'num_res_units': 1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/mnt/jonathan/2024_czii_mlchallenge_notebooks/pyUNET/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_23820/582407621.py", line 14, in <lambda>
    study.optimize(lambda trial: objective(trial, nclasses, train_loader, val_loader,
  File "/tmp/ipykernel_23820/3199552931.py", line 79, in objective
    score = train(train_loader, val_loader,
  File "/tmp/ipykernel_23820/3983403781.py", line 30, in train
    for batch_data in train_loader:
  File "/mnt/jonathan/2024_czii_mlchallenge_notebooks/pyUNET/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = s

KeyboardInterrupt: 